In [1]:
desk_jockeys = [{'name': 'Brandon',
                 'county': 'Lake',
                 'shift': 'AM'},
                {'name': 'David',
                 'county': 'Lake',
                 'shift': 'PM'},
                {'name': 'Brad',
                 'county': 'Marion',
                 'shift': 'Either'},
                {'name': 'Kody',
                 'county': 'Marion',
                 'shift': 'PM'},
                {'name': 'John',
                 'county': 'Hernando',
                 'shift': 'Either'},
                {'name': 'Rick',
                 'county': 'Hernando',
                 'shift': 'Either'},
                {'name': 'Ronan',
                 'county': 'Citrus',
                 'shift': 'AM'}]

def neighbors(i, lst, dist=1):
    llen = len(lst)
    if llen == 0:
        return([])
    elif i-dist < 0:
        return(lst[llen+i-dist:] + lst[0:i] + lst[i+1:i+1+dist])
    elif i+dist >= llen:
        return(lst[i-dist:i] + lst[i+1:llen] + lst[0:1+i+dist-llen])
    else:
        return(lst[i-dist:i] + lst[i+1:i+1+dist])

def deviation(lst):
    mean = sum(lst)/len(lst)
    devsum = 0
    for i in lst:
        devsum += (i-mean)**2

    return(devsum)

def eval_func(chromosome):
    score = 0.0
    for position, jockey in enumerate(chromosome):
        if jockey['county'] in [j['county'] for j in neighbors(position, chromosome)]:
            # Avoid hitting one county sequentially
            score += 0.5
        if jockey['name'] in [j['name'] for j in neighbors(position, chromosome, 2)]:
            # Don't cause people to have shifts too close to each other
            score += 0.4
        if jockey['shift'] == 'AM' and position % 2 != 0:
            # Honor shift preference
            score += 1.0
        elif jockey['shift'] == 'PM' and position % 2 == 0:
            score += 1.0

    return(score)


In [21]:
from math import exp
from random import random, shuffle
def accept_probability(e1, e2, temperature):
    if e2 > e1:
        #return(exp((e1-e2)/temperature))
        return(0.0)
    else:
        #return(0.8)
        return(1.0)

def anneal(state, energy, acceptance, kmax=10000000):
    wins = {}
    for k in range(1,kmax+1):
        temperature = kmax/k
        if k*10 % kmax == 0:
            print(int(100*k/kmax), [j['name'] for j in state], energy(state), temperature)
        neighbor = state[:]
        shuffle(neighbor)  # Using shuffle() basically defeats the purpose of the annealing.
        se = energy(state)
        ne = energy(neighbor)
        if accept_probability(se,
                              ne,
                              temperature) >= random():
            state = neighbor
            if ne == 0 and state not in wins:
                wins[state] = True
                print([j['name'] for j in state])


    return(state, energy(state))


In [22]:
#results = anneal(desk_jockeys+desk_jockeys,
#                 eval_func,
#                 accept_probability)
#
#print(100, [j['name'] for j in results[0]], results[1])

In [23]:
sum_jockeys = [{'name': 'Brandon',
                'county': 'Lake',
                'shift': 'AM'},
               {'name': 'David',
                'county': 'Lake',
                'shift': 'PM'},
               {'name': 'Jordan',
                'county': 'Sumter',
                'shift': 'either'},
               {'name': 'Kody',
                'county': 'Marion',
                'shift': 'AM'},
               {'name': 'Spencer',
                'county': 'Marion',
                'shift': 'PM'},
               {'name': 'Jordan',
                'county': 'Sumter',
                'shift': 'either'},
               {'name': 'John',
                'county': 'Hernando',
                'shift': 'PM'},
               {'name': 'Rick',
                'county': 'Hernando',
                'shift': 'AM'},
               {'name': 'Jordan',
                'county': 'Sumter',
                'shift': 'either'},
               {'name': 'Ronan',
                'county': 'Citrus',
                'shift': 'either'}]

def sum_eval_func(chromosome):
    score = 0.0
    for position, jockey in enumerate(chromosome):
        if jockey['name'] == 'Jordan':
            # Because Sumter is low workload, it gets all the exceptions
            # We just need to make sure it doesn't get a full day of shift
            if position % 2 == 0 and chromosome[position-1]['name'] == 'Sumter':
                score += 0.8
            elif position % 2 != 0 and chromosome[position+1]['name'] == 'Sumter':
                score += 0.8
        ### No longer true: John's back in Hernando
        #if jockey['name'] == 'John' and 'Sumter' in [j['county'] for j in neighbors(position, chromosome)]:
        #    # John's most often in Sumter lately, so let's avoid doubling up on him
        #    score += 0.5
        if jockey['county'] in [j['county'] for j in neighbors(position, chromosome)]:
            # Avoid hitting one county sequentially
            score += 0.5
        if jockey['name'] in [j['name'] for j in neighbors(position, chromosome, 2)]:
            # Don't cause people to have shifts too close to each other
            score += 0.4
        if jockey['shift'] == 'AM' and position % 2 != 0:
            # Honor shift preference
            score += 10.0
        elif jockey['shift'] == 'PM' and position % 2 == 0:
            score += 10.0

    return(score)


results = anneal(sum_jockeys,
                 eval_func,
                 accept_probability)

print(100, [j['name'] for j in results[0]], results[1])

TypeError: unhashable type: 'list'